In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print('Groq API Key loaded successfully')
else:
    print('Groq API Key loading failed, please make sure the .env file exists and the spelling is correct')

if os.getenv("TAVILY_API_KEY"):
    print('Tavily API Key loaded successfully')
else:
    print('Tavily API Key loading failed, please make sure the .env file exists and the spelling is correct')

Groq API Key loaded successfully
Tavily API Key loaded successfully


In [2]:
import re
import httpx
import os

from langchain_groq import ChatGroq
load_dotenv()

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [67]:

# Import Autogen components
from autogen import AssistantAgent, UserProxyAgent,  ConversableAgent,register_function
from typing import TypedDict, List, Annotated
import operator
from typing import TypedDict, List, Tuple
import operator



In [50]:
llm_config = {"model": "llama-3.1-70b-versatile", "api_key": os.getenv("GROQ_API_KEY"), "api_type": "groq"}
assistant = ConversableAgent("chatbot", llm_config=llm_config)


In [51]:
reply = assistant.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)


>>>>>>>> USING AUTO REPLY...
{'content': 'A man walked into a library and asked the librarian, "Do you have any books on Pavlov\'s dogs and Schrödinger\'s cat?" \n\nThe librarian replied, "It rings a bell, but I\'m not sure if it\'s here or not."', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}


In [41]:
import os

player_config_list = [
    {
        "model": "llama-3.1-70b-versatile",
        "api_key": os.environ.get("GROQ_API_KEY"),
        "api_type": "groq"
    },
]


In [70]:
player_red = ConversableAgent(
    name="Player Red",
    system_message="You are a code names player. "
    "First call get_grid() first, to get list of words of your team. "
    "Then call make_move(move) to make a move.",
    llm_config={"config_list": player_config_list, "cache_seed": None},
)

player_blue = ConversableAgent(
    name="Player Blue",
    system_message="You are a code names player. "
    "First call get_grid() first, to get list of words of your team. "
    "Then call make_move(move) to make a move.",
    llm_config={"config_list": player_config_list, "cache_seed": None},
)

In [71]:
def get_grid()-> Annotated[str, "A list of words for associated team on the grid"]:
  
    return  "default"

def make_move(move: Annotated[str, "A move in form of a word,int pair for the team to guess."]) -> Annotated[str, "Result of the move."]:
    return " words to uncover in grid"

In [44]:
def check_made_move(msg):
    global made_move
    if made_move:
        made_move = False
        return True
    else:
        return False


grid_proxy = ConversableAgent(
    name="Board Proxy",
    llm_config=False,
    # The board proxy will only terminate the conversation if the player has made a move.
    is_termination_msg=check_made_move,
    # The auto reply message is set to keep the player agent retrying until a move is made.
    default_auto_reply="Please make a move.",
    human_input_mode="NEVER",
)

In [72]:

register_function(
    make_move,
    caller=player_red,
    executor=grid_proxy,
    name="make_move",
    description="Call this tool to make a move.",
)

register_function(
    get_grid,
    caller=player_red,
    executor=grid_proxy,
    name="get_grid",
    description="Get grid and words for each team.",
)

register_function(
    make_move,
    caller=player_blue,
    executor=grid_proxy,
    name="make_move",
    description="Call this tool to make a move.",
)

register_function(
    get_grid,
    caller=player_blue,
    executor=grid_proxy,
    name="get_grid",
    description="Get grid and words for each team.",
)

/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/autogen/agentchat/conversable_agent.py:2504: UserWarning: Function 'make_move' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)
/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/autogen/agentchat/conversable_agent.py:2504: UserWarning: Function 'get_grid' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


In [74]:
llm_config={"config_list": player_config_list, "cache_seed": 42}

member1 = ConversableAgent(
    name="Team member1",
    system_message="Short in answer.",
    llm_config=llm_config,
)
member2 = ConversableAgent(
    name="Team member2",
    system_message="Creative but short in answer.",
    llm_config=llm_config,
)

In [73]:
from autogen import GroupChatManager, GroupChat

groupchat = GroupChat(agents=[  member1, member2], messages=[], max_round=3, speaker_selection_method='round_robin',  # Ensures turn-taking
    allow_repeat_speaker=False )
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [75]:

initial_message="find the words on the given grid given the clue by your spymaster."

initializer = UserProxyAgent(
    name="Init",
)

try:
    initializer.initiate_chat(manager, message =initial_message)
except Exception as e:
    print(f"An error occurred during chat initiation: {e}")


Init (to chat_manager):

find the words on the given grid given the clue by your spymaster.

--------------------------------------------------------------------------------

Next speaker: Team member1


>>>>>>>> USING AUTO REPLY...
Team member1 (to chat_manager):

I'd be happy to help you find the words on the grid. However, I need a few more details. 

Please provide me with the following:

1. The grid itself: I need the letters on the grid to be able to help you.
2. The clue from your spymaster: I need to know what the clue is so I can give you accurate answers.
3. Any other specific instructions or context: Are there any specific rules or constraints I should be aware of?

Once I have this information, I'll do my best to help you find the words on the grid.

--------------------------------------------------------------------------------

Next speaker: Team member2


>>>>>>>> USING AUTO REPLY...
Team member2 (to chat_manager):

Here's the grid and other details:

Grid:
A E H L N
O 